In [1]:
import re
import numpy as np
import spacy
import pandas as pd
import nltk
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk import pos_tag

In [2]:
INTENSADV1={'absolutely', 'absurdly', 'resoundingly', 'amazingly', 'awfully', 'extremely', 'completely', 'highly', 'incredibly', 'perfectly', 'quite', 'really', 'strikingly', 'surprisingly', 'terribly', 'totally','unbelievably','hugely', 'unnaturally', 'unusually', 'utterly', 'very', 'tremendously', 'spectacularly'}
INTENSADJ1={'absolute', 'extreme', 'incredible', 'perfect', 'phenomenal', 'spectacular', 'huge', 'major', 'tremendous', 'complete', 'considerable', 'real', 'terrible', 'total', 'unbelievable', 'utter', 'great', 'resounding'}

In [3]:
DYS1= {'uh','um', 'mm-hmm', 'uh-huh', 'huh' }

In [4]:
I1={'i', "i\'m", "i\'ve", "i\'ll", "i\'d",'me'}
YOU1={'you', "you\'re", "you\'ve","you\'ll", "you\'d"}
HE1={'he', "he\'s", "he\'ll", "he\'d",'him'}
SHE1={'she', "she\'s", "she\'ll", "she\'d",'her'}
WE1={'we', "we\'re", "we\'ll", "we\'d", "we\'ve",'us'}
THEY1={'they', "they\'re", "they\'ll", "they\'d", "they\'ve",'them'}
PRONSUBJ1={'i', 'you', 'he', 'she', 'it', 'we', 'they'}

In [5]:
#emo1=stative, relational, positive
#emo2=stative, relational, negative
EMO1V={'like', 'adore', 'want', 'prefer', 'love', 'enjoy'}
EMO1N={'like', 'adoration', 'want', 'preference', 'love', 'enjoyment'}
EMO2V={'hate', 'dislike', 'disprefer'}
EMO2N={'hate', 'dislike', 'dispreference'}

In [6]:
BE1={'be','is','am','are','were','was','been','being'}
BECL1={"he\'s","I\'m","it\'s","she\'s","they\'re","we\'re","you\'re", "that\'s"}
BENEG1={"aren\'t","isn\'t","wasn\'t","weren\'t", "am not", "is not", "are not", "was not", "were not"}
MODAL1={'might','may','could','can','should','shall','will','would'}
MAY1={'may','might'}
MAYNEG1={'may not','might not',"mightn\'t", "mayn\'t"}
MAYHAVE1={'may have', 'might have', "might\'ve", "may\'ve"}
MAYNEGHAVE1={"may not have", "might not have", "mightn\'t have", "mayn\'t have"}
CAN1={'can','could'}
CANNEG1={'cannot',"can\'t","couldn\'t", 'could not'}
CANHAVE1={'can have', 'could have', "could\'ve"}
SHALL1={'shall','should', 'shalt'}
SHALLHAVE1={'shall have', 'should have'}
SHALLNEG1={"shan't", "shouldn\'t", "shall not", "should not"}
SHALLNEGHAVE1={'shall not have', 'should not have', "shouldn\'t have"}
ABLE1={'able','unable'}
AINT1={"ain\'t"}
WOULDCL1={"you\'d","he\'d","I\'d","it\'d","she\'d","they\'d","we\'d", "that\'d", "this\'d"}
HAVECL1={"i\'ve", "you\'ve", "she\'s", "he\'s", "we\'ve", "they\'ve", "it\'s", "that\'s", "this\'s"}
WILL1={'will', 'would'}
WILLNEG1={"wouldn\'t","won\'t", 'will not', 'would not'}
WILLCL1={"i\'ll", "you\'ll", "she\'ll", "he\'ll","that\'ll", "this\'ll"}
DARE1={'dare'}
DARENEG1={"daren\'t", 'dare not'}
HAVE1={'have','has','had','having'}
HAVENEG1={"haven\'t","hasn\'t","hadn\'t", 'had not', 'has not', 'have not'}
HAVEGOTTA1={'has got to', 'has gotta', 'have got to', 'have gotta', 'gotta'}
HAVEGOTTANEG1={"hasn\'t got to", "hasn\'t gotta", "haven\'t got to", "haven\'t gotta", "ain\'t gotta", "ain\'t got to"}
HAVEGOTTACL1={"it\'s gotta", "it\'s got to", "he\'s got to", "he\'s gotta", "she\'s got to", "she\'s gotta", "i\'ve gotta", "i\'ve got to", "we\'ve gotta", "we\'ve got to", "they've got to", "they've gotta", "you\'ve got to", "you\'ve gotta", "that\'s got to", "that\'s gotta", "this\'s gotta", "this\'s got to"}
MUST1={'must'}
MUSTNEG1={"mustn\'t"}
MUSTHAVE1={'must have', "must\'ve"}
MUSTNEGHAVE1={'must not have', "mustn\'t have", "mustn\'t \'ve"}
USEDTO1={'used to'}
NEED1={'need'}
NEEDNEG1={"needn\'t"}
NEEDHAVE1={"need\'ve", 'need have'}
NEEDHAVENEG1={"needn\'t have", "needn\'t\'ve", 'need not have'}
OUGHT1={"ought to", 'oughta'}
OUGHTNEG1={"oughtn\'t to"}
OUGHTHAVE1={'ought to have', 'oughta have'}
DO1={'do', 'does'}
DONEG1={"doesn\'t","don\'t"}
GONNA1={'am going to', 'are going to', 'is going to', 'am gonna', 'are gonna', 'is gonna'}
GONNANEG1={"am not going to", 'are not going to', 'is not going to', 'am not gonna', 'are not gonna', 'is not gonna', "ain\'t gonna", "isn\'t gonna", "aren\'t gonna"}
GONNACL1={"i\'m going to", "they\'re going to", "she\'s going to", "it\'s going to", "we\'re going to", "i\'m gonna", "you\'re gonna", "i\'m gonna", "you\'re gonna", "he\'s gonna", "she\'s gonna", "it\'s gonna", "we\'re gonna", "they\'re gonna", "that\'s gonna"}
GONNANEGCL1={"i\'m not going to", "they\'re not going to", "she\'s not going to", "it\'s not going to", "we\'re not going to", "i\'m not gonna", "you\'re not gonna", "he\'s not gonna", "she\'s not gonna", "it\'s not gonna", "we\'re not gonna", "they\'re not gonna", "that\'s not gonna"}

In [7]:
def assessments(sentence):
  pattern1='((our|my) (opinion|understanding) (is|was) that)|(it (is|was) (our|my) (opinion|understanding) (that)?)|(in (our|my) opinion)|((our|my) take on)|(it (seems|seemed) to (us|me) (that)?)|(it (seems|seemed) (that)?)|(it would seem to (us|me)?)|(it would appear to (us|me)?)|(it appears to (us|me)?)|((the|my|our) ([\w]+[ ])?point is (that)?)' 
  pattern2='((the|my|our) ([\w]+[ \,]*){1,2} point is (that)?)|(it (looks|looked) to (us|me) (as if|like))|(it (looks|looked) (as if|like|that way))|((we|i) (have|get|got) the impression (that)?)|((our|my) impression (was|is) (that)?)|(in (our|my) book)|(to (our|my) mind)|(to (our|my) way of thinking)|(as far as (I am|I was|we are|we were) concerned)'
  pattern3='(if you ask (me|us))|((our|my) feeling (is|was|would be))|(from where (I\'m|I am) (standing|sitting))|((we|I) (don\'t)? think (that)?)|(all (we\'re|I\'m) saying is)|(what (I\'m|we\'re) saying is)|((we\'re|I\'m) (not)? saying that)|(what (we\'re|i\'m) trying to say is)|(what (we|i) mean is (that)?)'
  m1=re.search(pattern1,sentence)
  m2=re.search(pattern2,sentence)
  m3=re.search(pattern3,sentence)
  if ((m1 is None) and(m2 is None)) and (m3 is None) :
    return 0
  else:
    return 1

In [8]:
def authority(sentence):
  pattern='(according to)'
  match=re.search(pattern,sentence)
  if match is None:
    return 0
  else:
    return 1

In [9]:
def causation(sentence):
  pattern1='(so|therefore|because|hence|as a result|consequently)'
  m=re.search(pattern1,sentence,re.IGNORECASE) 
  if m is None:
    return 0
  else:
    return 1

In [12]:
def conditionals(sentence):
  pattern1='(if (we|you) want to ([\w]+[ \,]+){1,7}(we|you) (need to|must|have to))|((we|you) ([\w ,]+) (must|have to|need to) ([\w]+[ \,]+){1,7}if  (you|we) want to)|(it would be ([\w]+[ \,]+){0,2}nice if)|(wouldn\'t it be ([\w]+[ \,]+){0,2}nice if)|(if ([\w]+[ \,]+){3,8} that would be ([\w]+[ \,]+){0,2}nice)|((cannot|will not|won\'t|can\'t) ([\w]+[ \,]+){1,7}(if|unless))'
  pattern2='((if|unless) ([\w]+[ \,]+){3,10}(cannot|will not|won\'t|can\'t))|((need|needs|must|has to|have to) ([\w]+[ \,]+){3,10}(in order )to)|((in order )?to ([\w]+[ \,]+){3,10}(need|needs|must|has to|have to))|(as long as (we|you) ([\w]+[ \,]+){3,10}(will|can|able|should|[a-zA-Z]+\'ll))|(([a-zA-Z]\'ll|will|can|able|should) ([\w]+[ \,]+){3,10}as long as (we|you))|((you|he|we) better ([\w]+[ \,]+){3,10}or)|(otherwise)'
  if ((re.search(pattern1,sentence)is None) and (re.search(pattern2,sentence) is None)):
    return 0
  else:
    return 1

In [14]:
def contrast(sentence):
  pattern1='(really)|(actually)|(as opposed to)|(instead of)|(rather than)|(there (are|is) ([\w]+[ \,]*){1,4} and (then )?there (are|is))|((is|that\'s|it\'s) a whole nother issue)|((is|are|that\'s|it\'s) (very|quite|completely|totally )?different)|(whole new ballgame)|((is|that\'s|it\'s) a (separate|different) (issue|question))'
  if re.search(pattern1,sentence) is None:
    return 0
  else:
    return 1

In [16]:
def difficulty(sentence):
  sentence=sentence.lower()
  pattern1="\b(((BE1) (INTENSADV1)?easy)|((BE1) a (INTENSADJ1)?breeze)|((BE1) a (INTENSADJ1)?walk in the park))|(((BE1) a (INTENSADJ1)?piece of cake)|((BE1) a (INTENSADJ1)?snap) |((BE1) a (INTENSADJ1)?cinch))|(((BE1) (INTENSADJ1)?child's play)|((BE1) (INTENSADV1)?difficult)|((BE1) a (INTENSADJ1)?pain)|((BE1) a (INTENSADJ1)?pain in the (butt|neck|ass)))|(((BE1) a (INTENSADJ1)?(bitch|bastard) to)|((BE1) no picnic)|((BE1) (INTENSADV1)?tricky)|((BE1) (INTENSADV1)?arduous))|(((BE1) a (INTENSADJ1)?challenge)|((BE1) (INTENSADV1)?challenging)|((HAVE1) a (INTENSADV1)?(hard|difficult) time))\b"
  if re.search(pattern1,sentence,re.IGNORECASE):
    return 0
  else:
    return 1


In [18]:
def doubt(sentence):
  pattern1="((i am|i\'m) not (sure|convinced))"
  pattern2="(i (don\'t|can\'t|do not|cannot) see how)"
  pattern3="(it (is not|isn\'t) (clear|evident|obvious) (that)?)"
  pattern4="(it\'s not (clear|evident|obvious) (that)?)"
  pattern5='((we|i) doubt (that)?)|((we|i) (am|are) doubtful)'
  pattern6="((we\'re|i\'m) doubtful)"

  if (((re.search(pattern1,sentence)is None) and (re.search(pattern2,sentence) is None)) and ((re.search(pattern3,sentence)is None) and (re.search(pattern4,sentence) is None)))and ((re.search(pattern5,sentence)is None) and (re.search(pattern6,sentence) is None)):
    return 0
  else:
    return 1

In [19]:
def emphasis(sentence):
  pattern1='(clearly|obviously|patently)|(when you (really )?think about it)|((it is|it\'s) ((really|pretty) )?(obvious|evident|clear) (that)?)|(definitely|(i have to say))'
  pattern2="((i\'ve got to say)|(i\'ve gotta say))"
  pattern3='((i should say)|surely|(for sure))|((BE1) ((sure)|(certain)|(confident)) (that)?)|((of course)|(no doubt about it)|(doubtless)|(without a doubt))|((I have no doubt (that)?)|(I bet (that)?)|((BE1) bound to))|(no two ways about it)|(there ((is)|(are)) no two ways about it)'
  pattern4='(there\'s no two ways about it)|(((the)|(one)) ((thing)|(issue)|(question)|(problem)) (MODAL1 )?(BE1) (that)?)|((my feeling is (that)?)|(that\'s why)|(that is why)|(the idea (here )?is (that)?))(((my)|(the)) whole ((point)|(question)) is )'
  pattern5='(what you have to do is)|(the reason is (that)? )|((GONNA1)|(GONNANEG1)|(GONNANEGCL1)|(GONNACL1))|(i want to (highlight|emphasize|underscore))'   
  pattern6="((here\'s what)|(here is what)|exactly|precisely)"
  pattern7="((what will happen is)|(what\'ll happen is))"
  pattern8="((what\'s ((gonna)|(going to)) happen is)|(what is ((gonna)|(going to)) happen is))"
  if (((re.search(pattern1,sentence)is None) and (re.search(pattern2,sentence) is None)) and ((re.search(pattern3,sentence)is None) and (re.search(pattern4,sentence) is None))) and (((re.search(pattern5,sentence)is None) and (re.search(pattern6,sentence) is None)) and ((re.search(pattern7,sentence)is None) and (re.search(pattern8,sentence) is None)) ):
    return 0
  else:
    return 1


In [21]:
def generalization(sentence):
  pattern1='((everybody|everything|anybody|anything|nobody|nothing))|((else|at all)|(in the (world|universe))|(of all times)|(in recent memory)|(in living history))'
  if re.search(pattern1,sentence) is None:
    return 0
  else:
    return 1

In [23]:
def inconsistency(sentence):
  pattern1='((except that)|(except for)|(with the exception of)|however|nevertheless)|((that said)|(that having been said)|(that being said)|despite|(in spite of)|(even so))|((at the same time)|still|(wait a minute)|(hold on a second)|(hold on a sec))|((it\'s just that)|(all well and good)|(as far as it goes)|(you might think (that)?)|(you may think (that)?))'
  if re.search(pattern1,sentence)is None:
    return 0
  else:
    return 1


In [25]:
def inyourshoes(sentence):
  pattern1="(what i would do)|(if i were you)"
  pattern2="(i would not)|(i wouldn\'t)"
  if ((re.search(pattern1,sentence)is None) and (re.search(pattern2,sentence) is None)):
    return 0
  else:
    return 1

In [27]:
def necessity(sentence):
  pattern1="\b((a must)|must|essential|indispensable|necessary)|(((BE1) a necessity)|needed|required|requirement)|((can\'t do without)|(got to)|gotta |(had better))|(hafta|(have to)|(has to)|(need to)|(needs to))|((ought to)|oughta|should|((PRONSUBJ1) better))|((necesssitates|necessitated|necessitating|necessitate))\b"
  if re.search(pattern1,sentence,re.IGNORECASE):
    return 0
  else:
    return 1

In [29]:
def possibility(sentence):
  pattern1="\b((you can)|(we can)|(you can\'t)|(you cannot)|(we can\'t))|((there\'s no way (that|for|of|to)?)|(any way (that|for|of|to)?)|(no way))|((we cannot)|(you could)|(we could)|((BE1) able to))\b"
  if re.search(pattern1,sentence,re.IGNORECASE):
    return 0
  else:
    return 1
  

In [31]:
def priority(sentence):
  pattern1='(important|crucial|key|essential|critical)|(fundamental|key|major|vital|(first and foremost))|(((now )?remember (that)?)|(keep in mind (that)?))'
  pattern2="(don\'t forget (that)?)"
  pattern3="(let\'s not forget)"
  pattern4="(let\'s keep in mind)"
  pattern5="(let\'s remember)"
  if  ((re.search(pattern1,sentence)is None) and (re.search(pattern2,sentence) is None)) and (re.search(pattern3,sentence)is None) and ((re.search(pattern4,sentence)is None) and (re.search(pattern5,sentence) is None)) :
    return 0
  else:
    return 1

In [33]:
def rhetoricalquestion(sentence):
  sentence=sentence.lower()
  pattern1='(do (we|you) (actually|really|still) (need|want))|(why not)'
  pattern2="(why don\'t (we|you))"
  pattern3="(what if)|((and )?who (wouldn\'t|doesn\'t) (EMO1V1))"
  if ((re.search(pattern1,sentence)is None) and (re.search(pattern2,sentence) is None)) and (re.search(pattern3,sentence)is None) :
    return 0
  else:
    return 1

In [35]:
def structure(sentence):
  sentence=sentence.lower()
  pattern1='(first|secondly|(first place)|(in the first place)(first of all))'
  if re.search(pattern1,sentence) is None:
    return 0
  else:
    return 1

In [37]:
def wants(sentence):
  pattern1='((you|we|i) (don\'t )?(want|wanna))'
  pattern2='((you|we|i) might (not )?(want|wanna))'
  if ((re.search(pattern1,sentence)is None) and (re.search(pattern2,sentence) is None)):
    return 0
  else:
    return 1

In [39]:
with open('/content/drive/MyDrive/detection.txt') as f:
    contents = f.read()
    print(contents)

There is evil and suffering in the world, including in Ukraine, yet religions claim God loves us but won’t do anything to help us because we have free will and we choose to let the child die. So God watches the child burn to death and does nothing. This does not necessarily mean there is no God but it does question the many claims made by religions about God and if it exists.
Because God could remove that evil/suffering but chooses not to, we can only conclude then God is not all-loving. If God wants to remove that evil from the world but cannot, then why do we call him all-powerful? In any case, God is not truly God and there's nothing to be believe a God does exist.
Some explain away the problem of evil by suggesting that we do not know God’s plan and we cannot see the ultimate good that might result from evil acts so apparently the child will die a horrible death according to God’s plan. I do not believe this explanation is believable.
Yes, this may be an extreme example but given t

In [40]:
nlp = spacy.load('en')

text = '''Your text here'''
tokens = nlp(contents)

for sent in tokens.sents:
    print(sent)


There is evil and suffering in the world, including in Ukraine, yet religions claim God loves us but won’t do anything to help us because we have free will
and we choose to let the child die.
So God watches the child burn to death and does nothing.
This does not necessarily mean there is no God
but it does question the many claims made by religions about God and
if it exists.

Because God could remove that evil/suffering but chooses not to, we can only conclude then
God is not all-loving.
If God wants to remove that evil from the world but cannot, then why do we call him all-powerful?
In any case, God is not truly God and there's nothing to be believe a God does exist.

Some explain away the problem of evil by suggesting that we do not know God’s plan and we cannot see the ultimate good that might result from evil acts
so apparently the child will die a horrible death according to God
’s plan.
I do not believe this explanation is believable.

Yes, this may be an extreme example but giv

In [41]:
df=pd.DataFrame

In [42]:
[sentence for sentence in contents.split('.') if sentence !='']

['There is evil and suffering in the world, including in Ukraine, yet religions claim God loves us but won’t do anything to help us because we have free will and we choose to let the child die',
 ' So God watches the child burn to death and does nothing',
 ' This does not necessarily mean there is no God but it does question the many claims made by religions about God and if it exists',
 '\nBecause God could remove that evil/suffering but chooses not to, we can only conclude then God is not all-loving',
 " If God wants to remove that evil from the world but cannot, then why do we call him all-powerful? In any case, God is not truly God and there's nothing to be believe a God does exist",
 '\nSome explain away the problem of evil by suggesting that we do not know God’s plan and we cannot see the ultimate good that might result from evil acts so apparently the child will die a horrible death according to God’s plan',
 ' I do not believe this explanation is believable',
 '\nYes, this may 

In [43]:
CoList = contents.split(".")
Counter=0  
for i in CoList:
    if i:
        Counter += 1
print(Counter)


1954


In [44]:
df=pd.DataFrame({'docs': [sentence for sentence in contents.split('.') if sentence !='']})

In [45]:
df

,docs
0,"There is evil and suffering in the world, incl..."
1,So God watches the child burn to death and do...
2,This does not necessarily mean there is no Go...
3,\nBecause God could remove that evil/suffering...
4,If God wants to remove that evil from the wor...
...,...
1949,I don’t think this is the appeal to and reaso...
1950,As many kids are taught to believe in their r...
1951,Many people may feel as though they can't lea...
1952,A lot of children who are raised from a young...


In [47]:
df['docs']=df.apply(lambda row: row['docs'].rstrip(),axis=1)

In [48]:
df['docs']=df.apply(lambda row: row['docs'].strip(),axis=1)

In [49]:
df['docs'] = df['docs'].str.lower()

In [50]:
df['docs'][3]

'because god could remove that evil/suffering but chooses not to, we can only conclude then god is not all-loving'

In [51]:
assessments(df['docs'][3])

0

In [52]:
df

,docs
0,"there is evil and suffering in the world, incl..."
1,so god watches the child burn to death and doe...
2,this does not necessarily mean there is no god...
3,because god could remove that evil/suffering b...
4,if god wants to remove that evil from the worl...
...,...
1949,i don’t think this is the appeal to and reason...
1950,as many kids are taught to believe in their re...
1951,many people may feel as though they can't leav...
1952,a lot of children who are raised from a young ...


In [53]:
df['docs'] = df['docs'].apply(str)

In [57]:
df['assessments']=df.apply(lambda row: assessments(row['docs']),axis=1)
df['authority']=df.apply(lambda row: authority(row['docs']),axis=1)
df['causation']=df.apply(lambda row: causation(row['docs']),axis=1)
df['conditionals']=df.apply(lambda row: conditionals(row['docs']),axis=1)
df['contrast']=df.apply(lambda row: contrast(row['docs']),axis=1)
df['difficulty']=df.apply(lambda row: difficulty(row['docs']),axis=1)
df['doubt']=df.apply(lambda row: doubt(row['docs']),axis=1)
df['emphasis']=df.apply(lambda row: emphasis(row['docs']),axis=1)
df['generalization']=df.apply(lambda row: generalization(row['docs']),axis=1)
df['inconsistency']=df.apply(lambda row: inconsistency(row['docs']),axis=1)
df['inyourshoes']=df.apply(lambda row: inyourshoes(row['docs']),axis=1)
df['necessity']=df.apply(lambda row: necessity(row['docs']),axis=1)
df['possibility']=df.apply(lambda row: possibility(row['docs']),axis=1)
df['priority']=df.apply(lambda row: priority(row['docs']),axis=1)
df['rhetoricalquestion']=df.apply(lambda row: rhetoricalquestion(row['docs']),axis=1)
df['structure']=df.apply(lambda row: structure(row['docs']),axis=1)
df['wants']=df.apply(lambda row: wants(row['docs']),axis=1)

In [58]:
df['arguing']=df['assessments']+df['authority']+df['causation']+df['conditionals']+df['contrast']+df['difficulty']+df['doubt']+df['emphasis']+df['generalization']+df['inconsistency']+df['inyourshoes']+df['necessity']+df['possibility']+df['priority']+df['rhetoricalquestion']+df['structure']+df['wants']

In [60]:
df['arguing']=df.apply(lambda row: 1 if row['arguing']>0 else 0,axis=1)

In [61]:
df['arguing'].value_counts()

0    1005
1     949
Name: arguing, dtype: int64

In [62]:
df=df[['docs','arguing']]

In [63]:
df

,docs,arguing
0,"there is evil and suffering in the world, incl...",1
1,so god watches the child burn to death and doe...,1
2,this does not necessarily mean there is no god...,0
3,because god could remove that evil/suffering b...,1
4,if god wants to remove that evil from the worl...,1
...,...,...
1949,i don’t think this is the appeal to and reason...,1
1950,as many kids are taught to believe in their re...,1
1951,many people may feel as though they can't leav...,1
1952,a lot of children who are raised from a young ...,1


In [64]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [65]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [66]:
def preprocess(text):
    text = text.lower()
    text_p = "".join([char for char in text if char not in string.punctuation])
    
    words = word_tokenize(text_p)
    
    stop_words = stopwords.words('english')
    filtered_words = [word for word in words if word not in stop_words]
    
    porter = PorterStemmer()
    stemmed = [porter.stem(word) for word in filtered_words]
    text_final = " ".join([char for char in stemmed])
    return text_final

In [67]:
df['docs1']=df['docs'].map(preprocess)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [68]:
df

,docs,arguing,docs1
0,"there is evil and suffering in the world, incl...",1,evil suffer world includ ukrain yet religion c...
1,so god watches the child burn to death and doe...,1,god watch child burn death noth
2,this does not necessarily mean there is no god...,0,necessarili mean god question mani claim made ...
3,because god could remove that evil/suffering b...,1,god could remov evilsuff choos conclud god alllov
4,if god wants to remove that evil from the worl...,1,god want remov evil world call allpow case god...
...,...,...,...
1949,i don’t think this is the appeal to and reason...,1,’ think appeal reason faith thought might comp...
1950,as many kids are taught to believe in their re...,1,mani kid taught believ religion question young...
1951,many people may feel as though they can't leav...,1,mani peopl may feel though cant leav religion ...
1952,a lot of children who are raised from a young ...,1,lot children rais young age believ religion ta...


In [69]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_counts = count_vect.fit_transform(df.docs1)
X_counts.shape

(1954, 3524)

In [70]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X_counts)
X_tfidf.shape

(1954, 3524)

In [71]:
X_train_counts=X_counts[:1500]
X_train_tfidf=X_tfidf[:1500]
Y_train=df.arguing[:1500]
X_test_counts=X_counts[1500:]
X_test_tfidf=X_tfidf[1500:]
Y_test=df.arguing[1500:]

In [72]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, Y_train)
predicted = clf.predict(X_test_tfidf)
print(np.mean(predicted == Y_test))

0.6960352422907489


In [74]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
clf=classifier.fit(X_train_tfidf, Y_train)
predicted_svm = clf.predict(X_test_tfidf)
print(np.mean(predicted_svm == Y_test))

0.6651982378854625


In [80]:
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=40)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train_tfidf, Y_train)

y_pred=clf.predict(X_test_tfidf)
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(Y_test, y_pred))

Accuracy: 0.7334801762114538


In [76]:
pip install lightgbm

In [85]:
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
import lightgbm 
from lightgbm import LGBMClassifier

In [86]:
parameters = {
    'num_leaves':[10,12,15,18,20,25], 
    'min_child_samples':[10,12,15,18,20,22,25,30,35,40],
    'max_depth':[-1,2,4,6,10,12],
    'learning_rate':[0.03,0.04,0.05,0.08,0.1,0.15,0.2],
    'reg_alpha':[0,0.01,0.03]}


lgb=LGBMClassifier()
clf=RandomizedSearchCV(lgb,parameters,scoring='accuracy')
clf.fit(X_train_tfidf, Y_train)
print(clf.best_params_)
predicted=clf.predict(X_test_tfidf)
print('Classification of the result is:')

{'reg_alpha': 0.03, 'num_leaves': 12, 'min_child_samples': 15, 'max_depth': 12, 'learning_rate': 0.08}
Classification of the result is:


In [87]:
model = LGBMClassifier(learning_rate= 0.08, max_depth=12, min_child_samples=15, num_leaves= 12,reg_alpha= 0.03,random_state=42)
model.fit(X_train_tfidf, Y_train,eval_set=[(X_test_tfidf,Y_test),(X_train_tfidf, Y_train)],verbose=10,eval_metric='auc')
print('Training accuracy {:.4f}'.format(model.score(X_train_tfidf, Y_train)))
print('Testing accuracy {:.4f}'.format(model.score(X_test_tfidf,Y_test)))

[10]	training's auc: 0.811261	training's binary_logloss: 0.562605	valid_0's auc: 0.725039	valid_0's binary_logloss: 0.610642
[20]	training's auc: 0.854493	training's binary_logloss: 0.507852	valid_0's auc: 0.753718	valid_0's binary_logloss: 0.580152
[30]	training's auc: 0.878689	training's binary_logloss: 0.472807	valid_0's auc: 0.773087	valid_0's binary_logloss: 0.564192
[40]	training's auc: 0.896544	training's binary_logloss: 0.446323	valid_0's auc: 0.774639	valid_0's binary_logloss: 0.558129
[50]	training's auc: 0.907678	training's binary_logloss: 0.425664	valid_0's auc: 0.777147	valid_0's binary_logloss: 0.553974
[60]	training's auc: 0.914815	training's binary_logloss: 0.408711	valid_0's auc: 0.772765	valid_0's binary_logloss: 0.553469
[70]	training's auc: 0.921693	training's binary_logloss: 0.393623	valid_0's auc: 0.780855	valid_0's binary_logloss: 0.551862
[80]	training's auc: 0.926335	training's binary_logloss: 0.380091	valid_0's auc: 0.773849	valid_0's binary_logloss: 0.553548
